In [1]:
# pip install pystac-client

from geoai.download import (
    download_naip,
    download_overture_buildings,
    extract_building_stats,
)

import pystac_client
import planetary_computer
import leafmap

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

In [2]:
bbox = (-76.15741548689954, 43.05635088078997, -76.15648427005196, 43.05692144640927)

In [3]:
m = leafmap.Map(center=[43.05635088078997, -76.15741548689954], zoom=16)
m.add_basemap("Google Satellite")
m

Map(center=[43.05635088078997, -76.15741548689954], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [4]:
time_range = "2020-12-01/2020-12-31"

search = catalog.search(collections=["ms-buildings"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

8

In [9]:
import geopandas as gpd
import folium
gdf = gpd.read_parquet(items)
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), 
                        gdf.geometry.centroid.x.mean()],
               zoom_start=10)

# 将 GeoDataFrame 转换为 GeoJSON 并添加到地图
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        'fillColor': '#ffff00',
        'color': '#000000',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)

NameError: name 'items' is not defined

In [ ]:
downloaded_files = download_naip(
    bbox=bbox,
    output_dir="naip_data",
    max_items=1,
    # year=2020,
)

print(f"Downloaded {len(downloaded_files)} files.")

In [ ]:
# Download buildings
data_file = download_overture_buildings(
    bbox=bbox,
    output_file="buildings.geojson",
    output_format="geojson",
    data_type="building",
    verbose=True,
)

In [ ]:
if data_file:
    stats = extract_building_stats(data_file)
    print(stats)

In [ ]:
import os
import sys
notebook_dir = os.getcwd()  # Gets current working directory
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(project_root)

import os
import leafmap


In [ ]:
m = leafmap.Map()
m.add_raster("naip_data/m_4711720_sw_11_060_20230701_20230911.tif", layer_name="NAIP")
m.add_geojson("buildings.geojson", layer_name="Buildings")
m